# Prepare DWH Schema and Tables

In [274]:
import numpy as np
import pandas as pd
import csv

## Define Helper Functions

In [275]:
# convert camel to snake
def convert_to_snake_case(text: str) -> str:
    text = text.strip()
    return ''.join(['_' + s.lower() if (s.isupper() and i > 0 and text[i - 1] != ' ' and not text[i - 1].isupper()) \
                    else '_' if s == ' ' \
                    else s.lower() for i, s in enumerate(text)])

# lower all string columns
def lower_all_object_columns_of_df(df: pd.DataFrame):
    string_columns = df.select_dtypes(include='object').columns
    df[string_columns] = df[string_columns].apply(lambda x: x.str.lower())

## Load Data

In [276]:
world_cup_players = pd.read_csv("../../data/input_data/world_cup_players.csv",
                            usecols=['MatchID', 'Team Initials', 'Player Name','Event'])

world_cup_matches = pd.read_csv("../../data/input_data/world_cup_matches.csv",
                            usecols=['MatchID', 'Year', 'Datetime', 'Stadium', 'City', 'Home Team Name','Home Team Goals', 'Away Team Goals', 'Away Team Name',
                                    'Attendance', 'Home Team Initials', 'Away Team Initials'])

world_cup_countries = pd.read_csv("../../data/input_data/fifa_countries_2006.csv",
                            usecols=['Position', 'Team'])

world_cup_players.columns = [convert_to_snake_case(col) for col in world_cup_players.columns]
world_cup_matches.columns = [convert_to_snake_case(col) for col in world_cup_matches.columns]
world_cup_countries.columns = [convert_to_snake_case(col) for col in world_cup_countries.columns]
lower_all_object_columns_of_df(world_cup_players)
lower_all_object_columns_of_df(world_cup_matches)
lower_all_object_columns_of_df(world_cup_countries)

In [277]:
world_cup_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   position  32 non-null     int64 
 1   team      32 non-null     object
dtypes: int64(1), object(1)
memory usage: 644.0+ bytes


In [278]:
world_cup_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   match_id       37784 non-null  int64 
 1   team_initials  37784 non-null  object
 2   player_name    37784 non-null  object
 3   event          9069 non-null   object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [279]:
world_cup_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                852 non-null    int64  
 1   datetime            852 non-null    object 
 2   stadium             852 non-null    object 
 3   city                852 non-null    object 
 4   home_team_name      852 non-null    object 
 5   home_team_goals     852 non-null    int64  
 6   away_team_goals     852 non-null    int64  
 7   away_team_name      852 non-null    object 
 8   attendance          850 non-null    float64
 9   match_id            852 non-null    int64  
 10  home_team_initials  852 non-null    object 
 11  away_team_initials  852 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 80.0+ KB


## Prepare Data and create Tables

In [280]:
# Prepare and create TeamDIM
world_cup_countries = world_cup_countries.rename(columns={'position': 'rank'})
world_cup_countries['team_id'] = range(1, len(world_cup_countries) + 1)

In [281]:
merged_matches_players = pd.merge(world_cup_matches, world_cup_players, on='match_id')
merged_matches_players_2006 = merged_matches_players[merged_matches_players['year'] == 2006].copy()
merged_matches_players_2006.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 28218 to 31160
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                2943 non-null   int64  
 1   datetime            2943 non-null   object 
 2   stadium             2943 non-null   object 
 3   city                2943 non-null   object 
 4   home_team_name      2943 non-null   object 
 5   home_team_goals     2943 non-null   int64  
 6   away_team_goals     2943 non-null   int64  
 7   away_team_name      2943 non-null   object 
 8   attendance          2943 non-null   float64
 9   match_id            2943 non-null   int64  
 10  home_team_initials  2943 non-null   object 
 11  away_team_initials  2943 non-null   object 
 12  team_initials       2943 non-null   object 
 13  player_name         2943 non-null   object 
 14  event               1023 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 367.9

In [282]:
# get missing char players 
players_with_special_char = merged_matches_players_2006[merged_matches_players_2006['player_name'].str.contains('�')]
players_with_special_char['player_name'].unique()

array(['uma�a m.', 'bola�os c.', 'nu�ez v.', 'c�ceres', 'acu�a', 'ca�iza',
       'nu�ez', 'caba�as', 'alvb�ge', 'k�llstr�m', 'allb�ck',
       'jo�o ricardo', 'andr� macanga', 'akw�', 'sim�o', 'z� kalanga',
       'loc�', 'lam�', 'fl�vio', 'm�rio', 'zuberb�hler', 'l�cio', 'kak�',
       'z� roberto', 'luis�o', 'ca�izares'], dtype=object)

In [283]:
# clean world_cup_players df and add data to
merged_matches_players_2006 = merged_matches_players_2006.rename(columns={'team_initials': 'player_team_initials'})
merged_matches_players_2006['player_match_id'] = range(1, len(merged_matches_players_2006) + 1)
merged_matches_players_2006['event'] = merged_matches_players_2006['event'].fillna('')
merged_matches_players_2006['red_cards'] = merged_matches_players_2006['event'].str.count('r')
merged_matches_players_2006['yellow_cards'] = merged_matches_players_2006['event'].str.count(r'(?<!rs)y')
merged_matches_players_2006['goals'] = merged_matches_players_2006['event'].str.count('g')
merged_matches_players_2006['player_name'] = merged_matches_players_2006['player_name'].replace(['uma�a m.', 'bola�os c.', 'nu�ez v.', 'c�ceres', 'acu�a', 'ca�iza',
       'nu�ez', 'caba�as', 'alvb�ge', 'k�llstr�m', 'allb�ck',
       'jo�o ricardo', 'andr� macanga', 'akw�', 'sim�o', 'z� kalanga',
       'loc�', 'lam�', 'fl�vio', 'm�rio', 'zuberb�hler', 'l�cio', 'kak�',
       'z� roberto', 'luis�o', 'ca�izares'], ['umaña M.', 'bolaños C.', 'nuñez V.', 'céceres', 'acuña', 'cañiza',
 'nuñez', 'cabañas', 'alvbäge', 'källström', 'allbäck',
 'joão ricardo', 'andrè macanga', 'akwá', 'simão', 'zé kalanga',
 'locó', 'lamá', 'flávio', 'mário', 'zuberbühler', 'lúcio', 'kaká',
 'zé roberto', 'luisão', 'cañizares'])

In [284]:
# get missing char matches 
matches_with_special_char = merged_matches_players_2006[merged_matches_players_2006[['home_team_name', 'away_team_name']].apply(lambda row: row.astype(str).str.contains('�')).any(axis=1)]
pd.Series(matches_with_special_char[['home_team_name', 'away_team_name']].values.ravel()).unique()

array(['argentina', "c�te d'ivoire", 'netherlands',
       'rn">serbia and montenegro'], dtype=object)

In [285]:
# clean world_cup_matches df and add data to
merged_matches_players_2006[['home_team_name', 'away_team_name']] = merged_matches_players_2006[['home_team_name', 'away_team_name']].replace('rn">', '', regex=True)
merged_matches_players_2006 = merged_matches_players_2006.replace("c�te d'ivoire", 'ivory coast')
merged_matches_players_2006 = merged_matches_players_2006.replace("korea republic", 'south korea')
merged_matches_players_2006 = merged_matches_players_2006.replace("ir iran", 'iran')
merged_matches_players_2006[['home_team_name', 'away_team_name']] = merged_matches_players_2006[['home_team_name', 'away_team_name']].replace('usa', 'united states')

# Create a mapping between old matchids and new matchids (starting from 1)
unique_ids = merged_matches_players_2006['match_id'].unique()
new_ids = range(1, len(unique_ids) + 1)
id_mapping = dict(zip(unique_ids, new_ids))
merged_matches_players_2006['match_id'] = merged_matches_players_2006['match_id'].map(id_mapping)

In [286]:
# add location id to main df
locationdim = merged_matches_players_2006.copy()
locationdim = locationdim.drop_duplicates(subset=['city', 'stadium'])
locationdim['location_id'] = range(1, len(locationdim) + 1)
locationdim = locationdim[['location_id', 'city', 'stadium']] 
merged_matches_players_2006 = pd.merge(merged_matches_players_2006, locationdim, on=['city', 'stadium'])

In [287]:
# add player id to main df
playerdim = merged_matches_players_2006.copy()
# all players in our dataset have different names, Júnior changed team, Asamoah changed shirt number
playerdim = playerdim.drop_duplicates(subset=['player_name'])
playerdim['player_id'] = range(1, len(playerdim) + 1)
playerdim = playerdim[['player_id', 'player_name']]
merged_matches_players_2006 = pd.merge(merged_matches_players_2006, playerdim, on=['player_name'])

In [288]:
home_matches = merged_matches_players_2006.merge(world_cup_countries, left_on='home_team_name', right_on='team', how='inner')
home_matches = home_matches[home_matches['home_team_initials'] == home_matches['player_team_initials']]
away_matches = merged_matches_players_2006.merge(world_cup_countries, left_on='away_team_name', right_on='team', how='inner')
away_matches = away_matches[away_matches['away_team_initials'] == away_matches['player_team_initials']]

# create final dataframe for fact table
merged_df = pd.concat([home_matches, away_matches])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2942
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  2943 non-null   int64  
 1   datetime              2943 non-null   object 
 2   stadium               2943 non-null   object 
 3   city                  2943 non-null   object 
 4   home_team_name        2943 non-null   object 
 5   home_team_goals       2943 non-null   int64  
 6   away_team_goals       2943 non-null   int64  
 7   away_team_name        2943 non-null   object 
 8   attendance            2943 non-null   float64
 9   match_id              2943 non-null   int64  
 10  home_team_initials    2943 non-null   object 
 11  away_team_initials    2943 non-null   object 
 12  player_team_initials  2943 non-null   object 
 13  player_name           2943 non-null   object 
 14  event                 2943 non-null   object 
 15  player_match_id       2943

In [289]:
# Prepare and create teamdim
teamdim = merged_df.copy()
teamdim = teamdim.drop_duplicates(subset='team_id')
teamdim = teamdim.rename(columns={'team_id': 'id'})
teamdim = teamdim[['id', 'team', 'rank']]
teamdim.to_csv('../../data/query_data/teamdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
teamdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 0 to 2897
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      32 non-null     int64 
 1   team    32 non-null     object
 2   rank    32 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ KB


In [290]:
# Prepare and create player_match_statisticsdim
player_match_statisticsdim = merged_df.copy()
player_match_statisticsdim = player_match_statisticsdim.rename(columns={'player_match_id': 'id'})
player_match_statisticsdim = player_match_statisticsdim[['id', 'goals', 'red_cards', 'yellow_cards']]
player_match_statisticsdim.to_csv('../../data/query_data/player_matchdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
player_match_statisticsdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2942
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            2943 non-null   int64
 1   goals         2943 non-null   int64
 2   red_cards     2943 non-null   int64
 3   yellow_cards  2943 non-null   int64
dtypes: int64(4)
memory usage: 115.0 KB


In [291]:
# Prepare and create matchdim
matchdim = merged_df.copy()
matchdim = matchdim.drop_duplicates(subset='match_id')
matchdim = matchdim.rename(columns={'match_id': 'id'})
matchdim = matchdim[['id', 'goals', 'red_cards', 'yellow_cards']]
matchdim.to_csv('../../data/query_data/matchdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
matchdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 0 to 2897
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            64 non-null     int64
 1   goals         64 non-null     int64
 2   red_cards     64 non-null     int64
 3   yellow_cards  64 non-null     int64
dtypes: int64(4)
memory usage: 2.5 KB


In [292]:
# Prepare and create locationdim
locationdim = merged_df.copy()
locationdim = locationdim.drop_duplicates(subset=['location_id'])
locationdim = locationdim.rename(columns={'location_id': 'id'})
locationdim = locationdim[['id', 'city', 'stadium']]
locationdim.to_csv('../../data/query_data/locationdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
locationdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 0 to 1563
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       12 non-null     int64 
 1   city     12 non-null     object
 2   stadium  12 non-null     object
dtypes: int64(1), object(2)
memory usage: 384.0+ bytes


In [293]:
# Prepare and create playerdim
playerdim = merged_df.copy()
playerdim = playerdim.drop_duplicates(subset='player_id')
playerdim = playerdim.rename(columns={'player_id': 'id'})
playerdim = playerdim[['id', 'player_name']]
playerdim.to_csv('../../data/query_data/playerdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
playerdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 736 entries, 0 to 2795
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           736 non-null    int64 
 1   player_name  736 non-null    object
dtypes: int64(1), object(1)
memory usage: 17.2+ KB


In [226]:
#TODO for creating the tables
# get unique times, get highest time from timedim pyexasol tabletime
# create full fact table